## OLS and IV via GMM
#### Author: Jaepil Lee
#### Date: 08/26/2020

The goal of this note is to see that OLS and IV are special cases of GMM.

1. GMM: Concepts and Algorithm
2. OLS via GMM
3. IV via GMM

In [1]:
ENV["COLUMNS"]=100; ENV["LINES"]=1000
# Importing packages...
using Pkg; Pkg.activate("./.."); Pkg.instantiate();
using Distributions, Statistics, Random, HypothesisTests,
      LinearAlgebra, Distances, SharedArrays, 
      Optim, ForwardDiff, NLSolversBase, 
      Plots, Printf, LaTeXStrings, PlotThemes,
      CSV, DataFrames, TimeSeries, Dates,
      FredApi, FredData;

  Activating project at `c:\Users\jaepi\OneDrive\Documents\GitHub\jaepillee0315.github.io`


## [How GMM works: OLS and LIV](#ols_liv_GMM)<a name="ols_liv_GMM_menu"></a>

Let's first see how GMM estimation routine works in OLS and LIV. Let's construct a hypothetical dataset with $N$ where:
- $y_i = \beta_0 + \beta_1 x_{i,1} + \beta_2 x_{i,2} + \varepsilon_i, \varepsilon_i \sim \mathcal{N}(0,\sigma^2)$ for $i=1,\ldots,N$
- $(\beta_0, \beta_1, \beta_2) = (0.15, 0.25, -1.5), \sigma = 1.2$

We all know what $\beta_{ols}$ looks like: $(X'X)^{-1}X'Y$. We also know its standard errors. Let's code it just for fun:

In [2]:
#%% DGP
function DGP_OLS(N, β_DGP, σ_DGP)
    Random.seed!(42) # RNG: We get the same number for replicability
    (β₀,β₁,β₂) = β_DGP
    x₁ = randn(N) # a Nx1 vector
    x₂ = randn(N) # a Nx1 vector
    ε = σ_DGP * randn(N)  # a Nx1 vector
    y = β₀ .+ β₁ .* x₁ .+ β₂ .* x₂ .+ ε # a Nx1 vector
    X = [ones(N, 1) x₁ x₂] # a Nx3 matrix
    # If you wish to use other statistical packages, use this CSV file.
    # df = DataFrame(X0 = X[:, 1], X1 = X[:, 2], X2 = X[:, 3], y = y);
    # CSV.write("./OLS.csv",df);
    return y, X
end
#%% OLS estimation
function OLS_estimation(y, X)
    N = length(y)
    β_OLS = inv(X'X) * X'y # a 3x1 vector given DGP
    ε_OLS = y - X * β_OLS
    invXX = inv(X'*X)
    Xε = X.*ε_OLS
    Σ_OLS = invXX*(Xε'*Xε)*invXX # V_HC0 - White-Heteroskedasticity-Robust VCov
    σ_OLS = sqrt.(diag(Σ_OLS))
    return β_OLS, σ_OLS
end;

Let's take $\mathbf{Y}$ and $\mathbf{X}$ we generated above and find $\beta$ via GMM. You will see that they are exactly equal to each other. Let $f_i(\beta):=f(Y_i,X_i,\beta)$ be the function of orthogonality conditions evaluated at $i$'s covariates. The orthogonality condition for OLS is $\mathbb{E}(\varepsilon_i X_i) = 0$, where $\varepsilon_i = Y_i - X_i'\beta$ and the expectation is taken over $i$. In the GMM framework, we wish to find $\beta$ that satisfies:
$$
    0 = \frac{1}{N}\sum_{i=1}^{N}f_i(\beta) = \frac{1}{N}\sum_{i=1}^{N} \varepsilon_i X_i= \frac{1}{N}\sum_{i=1}^{N} (Y_i - X_i'\beta) \times X_i.
$$
Notice there are $3 \times 1$ orthogonality conditions: $\varepsilon_i$ is a scalar and $X_i$ is a $3\times1$ vector. Thus, the above equation is an "element-wise" average of a vector-valued function.

If you recall from econometrics I, what OLS really does is minimizing the sum of squared residuals, $\frac{1}{N}\sum_{i=1}^N \varepsilon_i^2$ where $\varepsilon_i=Y_i-X_i'\beta$. The first order condition of the minimization problem is found by taking a gradient of the sum of squared residuals (as a function of $\beta$, $X$, and $Y$) with respect to $\beta$. 
$$
    \frac{\partial}{\partial\beta'}\left(\frac{1}{N}\sum_{i}^{N}Y_i^2 - \frac{1}{N}\sum_{i}^{N} 2 X_i Y_i \beta + \frac{1}{N}\sum_{i}^{N} \beta' X_i X_i' \beta \right) = \frac{1}{N}\sum_{i}^{N}2(X_iX_i'\beta - X_i Y_i) = 0 \\
    \frac{1}{N}\sum_{i}^{N}(Y_i - X_i'\beta) \times X_i = 0
$$
This is exactly equal to the moment that GMM uses.

In [3]:
# OLS via GMM
function GMM_OLS(y,X)

    #=
    f_OLS(β,y,X): orthogonality conditions
    W: weighting matrix. In OLS, this is the identity matrix.
    f_sum(β): a (k x 1) vector of sample average of normal function.
    obj_OLS(β): a "sandwich" of f_sum(β).
    Q: In OLS, it is inv(X'*X).
    Ω: In OLS, it is the outer product of (X'*ε).
    =#

    function f_OLS(β,y,X)
        return (y - X*β) .* X
    end
    
    k = size(X)[2]
    N = length(y)
    W = I;
    f_sum(β) = vec(sum(f_OLS(β, y, X),dims=1));
    obj_OLS(β) = N * (f_sum(β)/N)' * W * (f_sum(β)/N);
    opt = optimize(obj_OLS, randn(k), BFGS(), autodiff = :forward);

    β_OLS_GMM = opt.minimizer;
    Q = ForwardDiff.jacobian(β -> f_sum(β) , β_OLS_GMM)/N;
    Ω = f_OLS(β_OLS_GMM, y, X)'*f_OLS(β_OLS_GMM, y, X)/N;
    Σ_OLS_GMM = inv(Q'*W*Q)*(Q'*W*Ω*W*Q)*inv(Q'*W*Q)/N; # remember to divide by N to compute s.e. of β_hat!
    σ_OLS_GMM = sqrt.(diag(Σ_OLS_GMM));
    
    return β_OLS_GMM, σ_OLS_GMM

end;

In [4]:
#%% Simulation & Estimation
N = 100_000
β_DGP = (0.15,0.25,-1.5)
σ_DGP = 1.2
y_OLS, X_OLS = DGP_OLS(N, β_DGP, σ_DGP)
β_OLS, σ_OLS = OLS_estimation(y_OLS, X_OLS);
β_OLS_GMM, σ_OLS_GMM = GMM_OLS(y_OLS,X_OLS);

# Print the results
print("Using the formula for OLS:\n")
print("    |     β₀     |     β₁     |     β₂     | \n")
@printf("DGP | %10.7f | %10.7f | %10.7f |\n", β_DGP[1], β_DGP[2], β_DGP[3])
@printf("OLS | %10.7f | %10.7f | %10.7f |\n", β_OLS[1], β_OLS[2], β_OLS[3])
@printf("    |(%10.7f)|(%10.7f)|(%10.7f)|\n", σ_OLS[1], σ_OLS[2], σ_OLS[3])
@printf("GMM | %10.7f | %10.7f | %10.7f |\n", β_OLS_GMM[1], β_OLS_GMM[2], β_OLS_GMM[3])
@printf("    |(%10.7f)|(%10.7f)|(%10.7f)|\n", σ_OLS_GMM[1], σ_OLS_GMM[2], σ_OLS_GMM[3])
print("Standard errors in parentheses.")

Using the formula for OLS:
    |     β₀     |     β₁     |     β₂     | 
DGP |  0.1500000 |  0.2500000 | -1.5000000 |
OLS |  0.1574148 |  0.2556172 | -1.5032818 |
    |( 0.0038011)|( 0.0037832)|( 0.0038027)|
GMM |  0.1574148 |  0.2556172 | -1.5032818 |
    |( 0.0038011)|( 0.0037832)|( 0.0038027)|
Standard errors in parentheses.

As we expect, OLS and GMM estimation give us the same estimates.

Let's scale up a little bit and do LIV. 2SLS in the over-identified case is what we will use in GMM.

Let $f_i(\beta):=f(Y_i,X_i,Z_i,\beta)$ be the function of orthogonality conditions. The orthogonality condition for LIV is $\mathbb{E}(\varepsilon_i Z_i) = 0$, where $\varepsilon_i = Y_i - X_i'\beta$ and the expectation is taken over $i$. In the GMM framework, we wish to find $\beta$ that satisfies below:
$$
    0 = \frac{1}{N}\sum_{i=1}^{N}f_i(\beta) = \frac{1}{N}\sum_{i=1}^{N} \varepsilon_i Z_i= \frac{1}{N}\sum_{i=1}^{N} (Y_i - X_i'\beta) \times Z_i.
$$

If we have more instruments than the number of endogenous variables, we call the model is over-identified. In the example below, I set $x_2$ to be the endogenous variable and provide 2 instruments, $z_1$ and $z_2$. If we use both $z_1$ and $z_2$, then we have 4 orthogonality conditions and 3 parameters to estimate. It is almost surely impossible to find $\beta$ where the moment is zero. Thus, we just find $\beta$ that makes the moment as close to zero as possible. We instead can introduce some positive definite weighting matrix $W$ and make the estimate more efficient.

In [5]:
# DGP for LIV
function DGP_LIV(N, β_DGP,μ,Σ)
     Random.seed!(42)
     (β₀,β₁,β₂) = β_DGP
     (x₂, z₁, z₂, ε) = collect.(eachrow(rand(MvNormal(μ,Σ),N)))
     x₁ = randn(N)
     y = β₀ .+ β₁*x₁ .+ β₂*x₂ .+ ε # a Nx1 vector
     X = [x₁ x₂] # a Nx2 matrix
     Z = [z₁ z₂]
     return y, X, Z
end;

In [6]:
function LIV_estimation(y, X_data, Z)

     N = length(y)
     X      = [ones(N) X_data]
     X₁Z₁   = [ones(N) X[:,2] Z[:,1]]
     X₁Z₂   = [ones(N) X[:,2] Z[:,2]]
     X₁Z_oi = [ones(N) X[:,2] Z[:,1] Z[:,2]] # over-identified case
 
     # Naive OLS produces biased estimates
     β_IV_OLS  = inv(X'*X) * X'*y # a 3x1 vector
     ε_OLS = y - X * β_IV_OLS
     invXX = inv(X'*X)
     Xε = X.*ε_OLS
     Σ_IV_OLS = invXX*(Xε'*Xε)*invXX # V_HC0 - White-Heteroskedasticity-Robust VCov
     σ_IV_OLS = sqrt.(diag(Σ_IV_OLS))

     # Using Z₁ as instrument
     # β_IV_Z₁ = inv(X'*X₁Z₁)*(X₁Z₁'*y)
     # ε_IV_Z₁ = y - X * β_IV_Z₁
     # invXZ₁  = inv(X'*X₁Z₁)
     # Z₁ε     = X₁Z₁.*ε_IV_Z₁
     # Σ_IV_Z₁ = invXZ₁*(Z₁ε'*Z₁ε)*invXZ₁
     # σ_IV_Z₁ = sqrt.(diag(Σ_IV_Z₁))

     # Using Z₂ as instrument
     # β_IV_Z₂ = inv(X'*X₁Z₂)*(X₁Z₂'*y)
     # ε_IV_Z₂ = y - X * β_IV_Z₂
     # invXZ₂  = inv(X'*X₁Z₂)
     # Z₂ε     = X₁Z₂.*ε_IV_Z₂
     # Σ_IV_Z₂ = invXZ₂*(Z₂ε'*Z₂ε)*invXZ₂
     # σ_IV_Z₂ = sqrt.(diag(Σ_IV_Z₂))

     # 2SLS - Using Z₁ as instrument
     β_2SLS_Z₁  = inv(X'*X₁Z₁*inv(X₁Z₁'*X₁Z₁)*X₁Z₁'*X)*X'*X₁Z₁*inv(X₁Z₁'*X₁Z₁)*X₁Z₁'*y
     ε_2SLS_Z₁  = y - X * β_2SLS_Z₁
     Zε_2SLS_Z₁ = X₁Z₁.*ε_2SLS_Z₁
     Qxz₁  = X'*X₁Z₁
     Qz₁z₁ = X₁Z₁'*X₁Z₁
     Qz₁x  = X₁Z₁'*X
     Ω_2SLS_Z₁ = Zε_2SLS_Z₁'*Zε_2SLS_Z₁
     Σ_2SLS_Z₁ = inv(Qxz₁*inv(Qz₁z₁)*Qz₁x)*(Qxz₁*inv(Qz₁z₁)*Ω_2SLS_Z₁*inv(Qz₁z₁)*Qz₁x)*inv(Qxz₁*inv(Qz₁z₁)*Qz₁x)
     σ_2SLS_Z₁ =sqrt.(diag(Σ_2SLS_Z₁))

     # 2SLS - Using Z₂ as instrument
     # β_2SLS_Z₂  = inv(X'*X₁Z₁*inv(X₁Z₁'*X₁Z₁)*X₁Z₁'*X)*X'*X₁Z₁*inv(X₁Z₁'*X₁Z₁)*X₁Z₁'*y
     # ε_2SLS_Z₂  = y - X * β_2SLS_Z₂
     # Zε_2SLS_Z₂ = X₁Z₂.*ε_2SLS_Z₂
     # Qxz₂  = X'*X₁Z₂
     # Qz₂z₂ = X₁Z₂'*X₁Z₂
     # Qz₂x  = X₁Z₂'*X
     # Ω_2SLS_Z₂ = Zε_2SLS_Z₂'*Zε_2SLS_Z₂
     # Σ_2SLS_Z₂ = inv(Qxz₂*inv(Qz₂z₂)*Qz₂x)*(Qxz₂*inv(Qz₂z₂)*Ω_2SLS_Z₂*inv(Qz₂z₂)*Qz₂x)*inv(Qxz₂*inv(Qz₂z₂)*Qz₂x)
     # σ_2SLS_Z₂ =sqrt.(diag(Σ_2SLS_Z₂))

     # Using Z₁ and Z₂ as instrument: OVERIDENTIFIED!
     Qxz = X'*X₁Z_oi
     Qzz = X₁Z_oi'*X₁Z_oi
     Qzx = X₁Z_oi'*X
     β_2SLS_oi = inv(Qxz*inv(Qzz)*Qzx)*(Qxz*inv(Qzz)*X₁Z_oi')*y
     ε_2SLS_oi = y - X*β_2SLS_oi
     Zε = X₁Z_oi.*ε_2SLS_oi
     Ω_2SLS_oi = Zε'*Zε
     Σ_2SLS_oi= inv(Qxz*inv(Qzz)*Qzx)*(Qxz*inv(Qzz)*Ω_2SLS_oi*inv(Qzz)*Qzx)*inv(Qxz*inv(Qzz)*Qzx)
     σ_2SLS_oi=sqrt.(diag(Σ_2SLS_oi))

     # β_IV_Z₁,   σ_IV_Z₁,
     # β_IV_Z₂,   σ_IV_Z₂,
     # β_2SLS_Z₁, σ_2SLS_Z₁,
     # β_2SLS_Z₂, σ_2SLS_Z₂,

     return β_IV_OLS, σ_IV_OLS, β_2SLS_Z₁, σ_2SLS_Z₁, β_2SLS_oi, σ_2SLS_oi
end;

In [7]:
# order: x₂, Z₁, Z₂, ε
μ = [0.0; 0.0; 0.0; 0.0];
Σ = [2.0  0.5  -.5  1.0;
     0.5  1.0  0.5  0.0;
     -.5  0.5  1.0  0.0;
     1.0  0.0  0.0  2.0];
Y_LIV, X_LIV, Z_LIV = DGP_LIV(N, β_DGP,μ,Σ);
β_IV_OLS, σ_IV_OLS, β_2SLS_Z₁, σ_2SLS_Z₁, β_2SLS_oi, σ_2SLS_oi = LIV_estimation(Y_LIV, X_LIV, Z_LIV);

print("LIV result: β₂ is biased in OLS \n")
print("        |     β₀     |     β₁     |     β₂     | \n")
@printf("DGP     | %10.7f | %10.7f | %10.7f |\n", β_DGP[1], β_DGP[2], β_DGP[3])
@printf("OLS     | %10.7f | %10.7f | %10.7f |\n", β_IV_OLS[1], β_IV_OLS[2], β_IV_OLS[3])
@printf("        |(%10.7f)|(%10.7f)|(%10.7f)|\n", σ_OLS[1], σ_OLS[2], σ_OLS[3])
#@printf("IV-Z₁   | %10.7f | %10.7f | %10.7f |\n", β_IV_Z₁[1], β_IV_Z₁[2], β_IV_Z₁[3])
#@printf("        |(%10.7f)|(%10.7f)|(%10.7f)|\n", σ_IV_Z₁[1], σ_IV_Z₁[2], σ_IV_Z₁[3])
#@printf("IV-Z₂   | %10.7f | %10.7f | %10.7f |\n", β_IV_Z₂[1], β_IV_Z₂[2], β_IV_Z₂[3])
#@printf("        |(%10.7f)|(%10.7f)|(%10.7f)|\n", σ_IV_Z₂[1], σ_IV_Z₂[2], σ_IV_Z₂[3])
@printf("2SLS-Z₁ | %10.7f | %10.7f | %10.7f |\n", β_2SLS_Z₁[1], β_2SLS_Z₁[2], β_2SLS_Z₁[3])
@printf("        |(%10.7f)|(%10.7f)|(%10.7f)|\n", σ_2SLS_Z₁[1], σ_2SLS_Z₁[2], σ_2SLS_Z₁[3])
#@printf("2SLS-Z₂ | %10.7f | %10.7f | %10.7f |\n", β_2SLS_Z₂[1], β_2SLS_Z₂[2], β_2SLS_Z₂[3])
#@printf("        |(%7.4f)|(%7.4f)|(%7.4f)|\n", σ_2SLS_Z₂[1], σ_2SLS_Z₂[2], σ_2SLS_Z₂[3])
@printf("2SLS-oi | %10.7f | %10.7f | %10.7f |\n", β_2SLS_oi[1], β_2SLS_oi[2], β_2SLS_oi[3])
@printf("        |(%10.7f)|(%10.7f)|(%10.7f)|\n", σ_2SLS_oi[1], σ_2SLS_oi[2], σ_2SLS_oi[3])
print("Standard errors in parentheses.")

LIV result: β₂ is biased in OLS 
        |     β₀     |     β₁     |     β₂     | 
DGP     |  0.1500000 |  0.2500000 | -1.5000000 |
OLS     |  0.1479366 |  0.2514084 | -1.0057541 |
        |( 0.0038011)|( 0.0037832)|( 0.0038027)|
2SLS-Z₁ |  0.1467703 |  0.2517546 | -1.4979841 |
        |( 0.0044668)|( 0.0044696)|( 0.0088246)|
2SLS-oi |  0.1467379 |  0.2517642 | -1.5116826 |
        |( 0.0044974)|( 0.0045002)|( 0.0044776)|
Standard errors in parentheses.

You can see that LIV estimate for $\beta_2$ in over-identified case has smaller standard error compared to just-identified case. Motivated by these examples, let's take a look at what GMM does:

#### Definition ([Bruce Hansen, p.416](https://www.ssc.wisc.edu/~bhansen/econometrics/Econometrics.pdf))
Let $\beta$ be a $k\times1$ vector, $f_i(\beta)$ be a $l\times1$ orthogonality conditions, and $W$ be a $l\times l$ positive definite weighting matrix. Define the GMM criterion function $J(\beta)$ as
$$
    J(\beta) = N \left(\frac{1}{N}\sum_{i=1}^N f_i(\beta)\right)' W \left(\frac{1}{N}\sum_{i=1}^N f_i(\beta)\right).
$$
The GMM estimator is 
$$
    \hat{\beta}_{gmm} = \underset{\beta}{\arg\min}\; J(\beta).
$$

#### Proposition ([Bruce Hansen, p.432](https://www.ssc.wisc.edu/~bhansen/econometrics/Econometrics.pdf))
Under general regularity conditions, $\sqrt{N}(\hat{\beta}_{gmm}-\beta_0) \xrightarrow[]{d} N(0,V_\beta)$ where 
$$
\begin{align*}
    V_\beta &= (Q'WQ)^{-1}(Q'W\Omega WQ)(Q'WQ)^{-1}\\
    \Omega &= \mathbb{E}(f_i f_i') \\
    Q &= \mathbb{E}\left[\frac{\partial}{\partial \beta'}f_i(\beta)\right].
\end{align*}
$$
If $W$ is efficient, $V_\beta = (Q'\Omega^{-1} Q)^{-1}$.

Note that when you compute standard errors for $\hat{\beta}_{gmm}$, you must divide $V_\beta$ by $N$! $V_\beta$ is the variance for $\sqrt{N}(\hat{\beta}_{gmm}-\beta)$, not $\hat{\beta}_{gmm}$ .

#### Two-step GMM ([Bruce Hansen, p.444](https://www.ssc.wisc.edu/~bhansen/econometrics/Econometrics.pdf))
In order to construct the efficient GMM estimator, we need to know what is the optimal $W$ in our model. In LIV, we know that $W=(Z'Z)^{-1}$. If the optimal weighting matrix $W=\Omega^{-1}$ is unknown, then we can first estimate $W$ with a standard GMM routine (say, by setting $W=I$), and then do the GMM routine again with $W=\hat{\Omega}^{-1}$.

#### Programming
In this case, we can simply code GMM as follows:
1. Set some weighting matrix $W$. In our example, for the just-identified case, use $W=I$ and for the over-identified case, use $W=(Z'Z)^{-1}$.
1. Construct the GMM criterion function. It is **obj_IV** in my code.
1. Ask the computer to minimize the criterion function for you. The minimizer is your $\hat{\beta}_{gmm}$.
1. Compute $Q$ and $\Omega$.
    - If you are using a two-step GMM, then repeat step 3 and 4 by setting $W=\hat{\Omega}^{-1}$
1. Compute the asymptotic variance $V_\beta$. 
1. Compute standard errors with $\textbf{sqrt}.(\textbf{diag}(\mathbf{V_\beta}/N))$

In [8]:
#%% GMM_IV
function GMM_one_step(y,Z_data,X_data;W=I)

    #=
    f_IV(β, y, Z, X): orthogonality conditions
    W: the weighting matrix. In one-step GMM, we do not update this.
    =#
   
    N = length(y)
    X = [ones(N) X_data]
    Z = [ones(N) X_data[:,1] Z_data]
    k = size(X)[2] # size of β
    
    function f_IV(β, y, Z, X)
        return (y - X*β) .* Z # (N×1) * (N×4)
    end
    
    f_sum_IV(β) = vec(sum(f_IV(β, y, Z, X),dims=1));
    obj_IV(β) = N * (f_sum_IV(β)/N)' * W * (f_sum_IV(β)/N);
    opt = optimize(obj_IV, randn(k), BFGS(), autodiff = :forward);
    β_GMM = opt.minimizer
    Q = ForwardDiff.jacobian(β -> f_sum_IV(β) , β_GMM)/N;
    Ω = f_IV(β_GMM, y, Z, X)'*f_IV(β_GMM, y, Z, X)/N;
    V_GMM = inv(Q'*W*Q)*(Q'*W*Ω*W*Q)*inv(Q'*W*Q)/N; # remember to divide by N!
    σ_GMM = sqrt.(diag(V_GMM))

    return β_GMM, σ_GMM

end;

In [9]:
function GMM_two_step(y,Z_data,X_data)

    #=
    f_IV(β, y, Z, X): orthogonality conditions
    W: the weighting matrix. In this case, we update this once.
    =#

    N = length(y)
    X = [ones(N) X_data]
    Z = [ones(N) X_data[:,1] Z_data]
    k = size(X)[2] # size of β
    
    function f_IV(β, y, Z, X)
        return (y - X*β) .* Z # (N×1) * (N×4)
    end
    f_sum_IV(β) = vec(sum(f_IV(β, y, Z, X),dims=1));
    
    # First-step: the estimates and the standard errors are the same as setting W=I in one-step GMM.
    W_1 = I;
    obj_GMM_1(β) = N * (f_sum_IV(β)/N)' * W_1 * (f_sum_IV(β)/N);
    opt_1 = optimize(obj_GMM_1, randn(k), BFGS(), autodiff = :forward);
    β_GMM_1 = opt_1.minimizer
    Q_1 = ForwardDiff.jacobian(β -> f_sum_IV(β) , β_GMM_1)/N;
    Ω_1 = f_IV(β_GMM_1, y, Z, X)'*f_IV(β_GMM_1, y, Z, X)/N;

    # Second-step
    W_opt = inv(Ω_1)
    obj_GMM_2(β) = N * (f_sum_IV(β)/N)' * W_opt * (f_sum_IV(β)/N);
    opt_2 = optimize(obj_GMM_2, randn(k), BFGS(), autodiff = :forward);
    β_GMM_2 = opt_2.minimizer
    Q_2 = ForwardDiff.jacobian(β -> f_sum_IV(β) , β_GMM_2)/N;
    Ω_2 = f_IV(β_GMM_2, y, Z, X)'*f_IV(β_GMM_2, y, Z, X)/N;
    
    # Results
    V_GMM_1 = inv(Q_1'*W_1*Q_1)*(Q_1'*W_1*Ω_1*W_1*Q_1)*inv(Q_1'*W_1*Q_1)/N; # remember to divide by N!
    σ_GMM_1 = sqrt.(diag(V_GMM_1))
    V_GMM_2 = inv(Q_2'*W_opt*Q_2)*(Q_2'*W_opt*Ω_2*W_opt*Q_2)*inv(Q_2'*W_opt*Q_2)/N; # remember to divide by N!
    σ_GMM_2 = sqrt.(diag(V_GMM_2))

    return β_GMM_1, σ_GMM_1, β_GMM_2, σ_GMM_2

end;

In [10]:
# Estimation
Z = [ones(N) X_LIV[:,1] Z_LIV];
β_GMM_I,  σ_GMM_I      = GMM_one_step(Y_LIV,Z_LIV,X_LIV,W=I);
β_GMM_ZZ, σ_GMM_ZZ     = GMM_one_step(Y_LIV,Z_LIV,X_LIV,W=inv(Z'*Z));
_, _, β_GMM_2, σ_GMM_2 = GMM_two_step(Y_LIV,Z_LIV,X_LIV);

# Print the results
print("GMM result, overidentified: β₂ is biased in OLS \n")
print("              |     β₀    |     β₁    |     β₂    | \n")
@printf("DGP           | %10.7f | %10.7f | %10.7f |\n", β_DGP[1], β_DGP[2], β_DGP[3])
@printf("One-step GMM: | %10.7f | %10.7f | %10.7f |\n", β_GMM_I[1], β_GMM_I[2], β_GMM_I[3])
@printf(" W=I          |(%10.7f)|(%10.7f)|(%10.7f)|\n", σ_GMM_I[1], σ_GMM_I[2], σ_GMM_I[3])
@printf("One-step GMM: | %10.7f | %10.7f | %10.7f |\n", β_GMM_ZZ[1], β_GMM_ZZ[2], β_GMM_ZZ[3])
@printf(" W=inv(Z'Z)   |(%10.7f)|(%10.7f)|(%10.7f)|\n", σ_GMM_ZZ[1], σ_GMM_ZZ[2], σ_GMM_ZZ[3])
@printf("Two-step GMM  | %10.7f | %10.7f | %10.7f |\n", β_GMM_2[1], β_GMM_2[2], β_GMM_2[3])
@printf("              |(%10.7f)|(%10.7f)|(%10.7f)|\n", σ_GMM_2[1], σ_GMM_2[2], σ_GMM_2[3])
print("Standard errors in parentheses.")

GMM result, overidentified: β₂ is biased in OLS 
              |     β₀    |     β₁    |     β₂    | 
DGP           |  0.1500000 |  0.2500000 | -1.5000000 |
One-step GMM: |  0.1467573 |  0.2517024 | -1.5115486 |
 W=I          |( 0.0044970)|( 0.0045002)|( 0.0044782)|
One-step GMM: |  0.1467379 |  0.2517642 | -1.5116826 |
 W=inv(Z'Z)   |( 0.0044974)|( 0.0045002)|( 0.0044776)|
Two-step GMM  |  0.1467965 |  0.2517965 | -1.5117060 |
              |( 0.0044973)|( 0.0045002)|( 0.0044777)|
Standard errors in parentheses.

References:
- [Professor Miller's Lecture Notes](https://comlabgames.com/structuraleconometrics/)
- [Bruce Hansen's econometrics textbook](https://www.ssc.wisc.edu/~bhansen/econometrics/Econometrics.pdf)
- Hansen, Lars Peter, and Kenneth J. Singleton. "Generalized instrumental variables estimation of nonlinear rational expectations models." Econometrica: Journal of the Econometric Society (1982): 1269-1286.
- Hansen, Lars Peter. "Large sample properties of generalized method of moments estimators." Econometrica: Journal of the econometric society (1982): 1029-1054.
- Newey, Whitney K., and Daniel McFadden. "Large sample estimation and hypothesis testing." Handbook of econometrics 4 (1994): 2111-2245.